<a href="https://colab.research.google.com/github/yf591/sd-model-merge-tool/blob/main/04_Merge_Model_Maker_Ver1_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2モデルの階層マージ（Hugging Face用）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import output

# Hugging Face Hub, PyTorch, その他必要なライブラリをインストール
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # PyTorchを使用して深層学習モデルを操作します。CUDAバージョン（例: `cu118`）を指定
!pip install diffusers transformers accelerate # Stable Diffusionを扱うための主要ライブラリです。モデルのロードや画像生成の操作を簡素化
!pip install safetensors # 安全かつ軽量なモデル保存形式（`.safetensors`）をサポート
!pip install huggingface-hub # Hugging Face Hubからモデルをダウンロード・管理
!pip install opencv-python # 生成した画像の前処理や後処理に使用
!pip install numpy # 数値計算ライブラリで、モデルや画像の操作に使う
!pip install matplotlib # 生成された画像の可視化に使う
!pip install tqdm # プログレスバーの表示
!pip install optuna # ハイパーパラメータ最適化

output.clear()

In [ ]:
import os
import torch
from safetensors.torch import load_file, save_file
from transformers import AutoConfig, AutoModel
from huggingface_hub import hf_hub_download
from getpass import getpass

In [ ]:
# Hugging Faceで取得したTokenをこちらに貼る(トークンを非表示で入力)
HF_TOKEN = getpass("Hugging FaceのRead権限のあるHF Tokenを入力してください: ")

In [ ]:
#@title ### 関数の定義（設定）

# ヘルパー関数
def download_model(repo_id, filename, token):
    """Hugging Face Hubからモデルをダウンロード"""
    return hf_hub_download(repo_id=repo_id, filename=filename, token=token)

def merge_weights(model1, model2, alpha, layer_weights):
    """指定されたレイヤーで重みをマージする"""
    merged = {}
    skipped_layers = [] # スキップされたレイヤーを記録するリスト
    for key in model1.keys():
        layer_name = key.split(".")[0] # レイヤー名を取得

        if key in model2 and layer_name in layer_weights:
            layer_weight = layer_weights.get(layer_name, alpha)
            try:
                merged[key] = layer_weight * model1[key] + (1 - layer_weight) * model2[key]
            except Exception as e:
                print(f"Skipped key: {key}, Error: {e}")
                skipped_layers.append(key) # スキップされたレイヤーを記録
                merged[key] = model1[key]
        else:
            merged[key] = model1[key]
    if skipped_layers:
       print("Skipped layers:", skipped_layers) # スキップされたレイヤーのリストを表示
    return merged


def save_merged_model(merged_weights, output_path):
    """マージ済みモデルを保存"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    save_file(merged_weights, output_path)

In [ ]:
#@title ### UI設定

#@markdown ### マージするモデルを設定

# UI設定
repo1 = "sazyou-roukaku/chilled_remix" #@param {type:"string"}
file1 = "chilled_remix_v2.safetensors" #@param {type:"string"}
repo2 = "SG161222/Realistic_Vision_V2.0" #@param {type:"string"}
file2 = "Realistic_Vision_V2.0-fp16-no-ema-inpainting.safetensors" #@param {type:"string"}

#@markdown ### アウトプット先を指定
output_file = "/content/drive/MyDrive/merged_model/merged_model.safetensors" #@param {type:"string"}

In [ ]:
#@markdown ### マージ比率Alphaの設定
alpha = 0.75 #@param {type:"slider", min:0.0, max:1.0, step:0.001}

#@markdown ### レイヤーごとのマージ比率の設定
layer_weight_IN00 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN01 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN02 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN03 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN04 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN05 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN06 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN07 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN08 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN09 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN10 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_IN11 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}

layer_weight_M00 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}

layer_weight_OUT00 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT01 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT02 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT03 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT04 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT05 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT06 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT07 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT08 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT09 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT10 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}
layer_weight_OUT11 = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}


# レイヤーごとの比率設定（コメントアウトした部分はマージ比率Alphaを適用）
layer_weights = {
    "IN00": layer_weight_IN00,
    "IN01": layer_weight_IN01,
    # "IN02": layer_weight_IN02,
    # "IN03": layer_weight_IN03,
    # "IN04": layer_weight_IN04,
    # "IN05": layer_weight_IN05,
    # "IN06": layer_weight_IN06,
    # "IN07": layer_weight_IN07,
    # "IN08": layer_weight_IN08,
    # "IN09": layer_weight_IN09,
    # "IN10": layer_weight_IN10,
    # "IN11": layer_weight_IN11,
    # "M00": layer_weight_M00,
    # "OUT00": layer_weight_OUT00,
    # "OUT01": layer_weight_OUT01,
    # "OUT02": layer_weight_OUT02,
    # "OUT03": layer_weight_OUT03,
    # "OUT04": layer_weight_OUT04,
    # "OUT05": layer_weight_OUT05,
    # "OUT06": layer_weight_OUT06,
    # "OUT07": layer_weight_OUT07,
    # "OUT08": layer_weight_OUT08,
    # "OUT09": layer_weight_OUT09,
    "OUT10": layer_weight_OUT10,
    "OUT11": layer_weight_OUT11
}

In [ ]:
#@title ###実行
try:
    print("モデルのダウンロードを開始します...")
    path1 = download_model(repo1, file1, HF_TOKEN)
    path2 = download_model(repo2, file2, HF_TOKEN)

    print("モデルをロード中...")
    model1 = load_file(path1, device="cuda" if torch.cuda.is_available() else "cpu")
    model2 = load_file(path2, device="cuda" if torch.cuda.is_available() else "cpu")

    # ここにモデルのキーを出力するコードを追加
    print("model1 keys:", model1.keys())
    print("model2 keys:", model2.keys())

    print("モデルをレイヤーごとにマージ中...")
    merged_weights = merge_weights(model1, model2, alpha, layer_weights)

    print(f"マージされたモデルを保存します: {output_file}")
    save_merged_model(merged_weights, output_file)
    print("マージ完了！")

except Exception as e:
    print(f"エラーが発生しました: {e}")